In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# Start with importing the data from WHOI

## Use the code in https://github.com/jerabaul29/MachineOcean_WP1_WHOI

In [2]:
import sys
sys.path.append("../../MachineOcean_WP1_WHOI/")

In [3]:
import mo_whoi_data

In [4]:
from mo_whoi_data.load_data import load_data_xarray

***** Put the interpreter in UTC, to make sure no TZ issues


In [5]:
d = load_data_xarray.load_all_into_xarray(run_on_ppi=True)

all_filenames: [PosixPath('/lustre/storeB/project/IT/geout/machine-ocean/prepared_datasets/in_situ_data_WHOI/WHOI/Transfer_Endurance_7.mat'),
                PosixPath('/lustre/storeB/project/IT/geout/machine-ocean/prepared_datasets/in_situ_data_WHOI/WHOI/Transfer_SPURS1.mat'),
                PosixPath('/lustre/storeB/project/IT/geout/machine-ocean/prepared_datasets/in_situ_data_WHOI/WHOI/Transfer_Endurance_3.mat'),
                PosixPath('/lustre/storeB/project/IT/geout/machine-ocean/prepared_datasets/in_situ_data_WHOI/WHOI/Transfer_Pioneer_6.mat'),
                PosixPath('/lustre/storeB/project/IT/geout/machine-ocean/prepared_datasets/in_situ_data_WHOI/WHOI/Transfer_Endurance_8.mat'),
                PosixPath('/lustre/storeB/project/IT/geout/machine-ocean/prepared_datasets/in_situ_data_WHOI/WHOI/Transfer_Pioneer_8.mat'),
                PosixPath('/lustre/storeB/project/IT/geout/machine-ocean/prepared_datasets/in_situ_data_WHOI/WHOI/Transfer_Irminger_3.mat'),
                

# Get buoy name from the dict keys

In [8]:
buoys = []
for key in d.keys():
    path_strings = str(key).split('.')[0].split('_')
    if path_strings[-1].isnumeric(): 
        buoy = path_strings[-2] + '_' + path_strings[-1]
    else:
        buoy = path_strings[-1]
    buoys.append(buoy)
buoys

['Endurance_7',
 'SPURS1',
 'Endurance_3',
 'Pioneer_6',
 'Endurance_8',
 'Pioneer_8',
 'Irminger_3',
 'SPURS2',
 'SouthernOcean2',
 'Pioneer_5',
 'Pioneer_3',
 'Pioneer_7',
 'Endurance_4',
 'CLIMODE',
 'Irminger_7',
 'Endurance_9',
 'Pioneer_10',
 'Irminger_6']

In [9]:
key

PosixPath('/lustre/storeB/project/IT/geout/machine-ocean/prepared_datasets/in_situ_data_WHOI/WHOI/Transfer_Irminger_6.mat')

In [10]:
df_in_situ = d[key]
df_in_situ

,CE,CN,IR,Lv,P10,Pair,Q10,Qair,RH,RH10,...,shf,sigH,sigHDir,stress,ustar,yday,yyyy,datetime,lat,lon
0,0.058741,0.083610,339.925002,2474.768030,1017.944242,1019.167502,7.248714,7.267303,93.933302,94.014437,...,4.723805,1.21,106.608233,0.023803,0.138781,223.007639,2019,2019-08-11 00:10:59.996491,59.9337,-39.47378
1,0.110629,0.096598,297.949997,2475.081231,1017.797864,1019.020508,7.268667,7.284779,93.525552,93.591409,...,2.631602,1.06,101.552518,0.017449,0.118826,223.049306,2019,2019-08-11 01:10:59.999656,59.9337,-39.47378
2,0.139019,0.126999,348.530000,2475.050118,1017.764995,1018.986008,7.469067,7.480979,93.906953,93.964349,...,0.503419,1.12,98.317455,0.010006,0.089936,223.090972,2019,2019-08-11 02:11:00.000769,59.9337,-39.47378
3,0.187421,0.070342,337.279997,2475.037386,1017.700083,1018.920502,7.588145,7.597242,94.722052,94.793354,...,0.064363,1.20,101.308285,0.006871,0.074493,223.132639,2019,2019-08-11 03:10:59.998945,59.9337,-39.47378
4,0.140239,0.010441,327.305002,2475.172549,1017.320471,1018.540506,7.613828,7.622443,95.132553,95.198827,...,-0.067704,1.02,106.933136,0.007151,0.075996,223.174306,2019,2019-08-11 04:10:59.997584,59.9337,-39.47378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932,0.078067,-0.018678,273.325001,2491.943886,997.514969,998.760501,3.100641,3.165751,80.403552,79.658724,...,73.053205,4.88,204.795173,0.419266,0.576409,387.215972,2020,2020-01-22 05:10:59.952046,59.9337,-39.47378
2933,0.078067,-0.018678,269.544999,2491.941121,997.503489,998.747000,2.916729,2.990077,73.412752,72.353448,...,62.215094,5.03,206.919255,0.409373,0.569918,387.257639,2020,2020-01-22 06:10:59.975539,59.9337,-39.47378
2934,0.078067,-0.018678,282.610001,2491.938347,997.989629,999.240002,3.011154,3.093219,83.999203,82.909322,...,99.670435,4.80,196.948117,0.498005,0.627132,387.299306,2020,2020-01-22 07:11:00.006928,59.9337,-39.47378
2935,0.078067,-0.018678,258.055002,2491.916288,998.007711,999.254504,2.977888,3.049324,78.179452,77.277268,...,95.349818,5.02,209.921894,0.748459,0.769358,387.340972,2020,2020-01-22 08:10:59.967979,59.9337,-39.47378


In [11]:
df_in_situ['datetime'] = pd.to_datetime(df_in_situ['datetime'])
df_in_situ = df_in_situ.set_index('datetime')
df_in_situ

,CE,CN,IR,Lv,P10,Pair,Q10,Qair,RH,RH10,...,rhoair,shf,sigH,sigHDir,stress,ustar,yday,yyyy,lat,lon
datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-11 00:10:59.996491,0.058741,0.083610,339.925002,2474.768030,1017.944242,1019.167502,7.248714,7.267303,93.933302,94.014437,...,1.245421,4.723805,1.21,106.608233,0.023803,0.138781,223.007639,2019,59.9337,-39.47378
2019-08-11 01:10:59.999656,0.110629,0.096598,297.949997,2475.081231,1017.797864,1019.020508,7.268667,7.284779,93.525552,93.591409,...,1.244759,2.631602,1.06,101.552518,0.017449,0.118826,223.049306,2019,59.9337,-39.47378
2019-08-11 02:11:00.000769,0.139019,0.126999,348.530000,2475.050118,1017.764995,1018.986008,7.469067,7.480979,93.906953,93.964349,...,1.243051,0.503419,1.12,98.317455,0.010006,0.089936,223.090972,2019,59.9337,-39.47378
2019-08-11 03:10:59.998945,0.187421,0.070342,337.279997,2475.037386,1017.700083,1018.920502,7.588145,7.597242,94.722052,94.793354,...,1.242429,0.064363,1.20,101.308285,0.006871,0.074493,223.132639,2019,59.9337,-39.47378
2019-08-11 04:10:59.997584,0.140239,0.010441,327.305002,2475.172549,1017.320471,1018.540506,7.613828,7.622443,95.132553,95.198827,...,1.242030,-0.067704,1.02,106.933136,0.007151,0.075996,223.174306,2019,59.9337,-39.47378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-22 05:10:59.952046,0.078067,-0.018678,273.325001,2491.943886,997.514969,998.760501,3.100641,3.165751,80.403552,79.658724,...,1.268590,73.053205,4.88,204.795173,0.419266,0.576409,387.215972,2020,59.9337,-39.47378
2020-01-22 06:10:59.975539,0.078067,-0.018678,269.544999,2491.941121,997.503489,998.747000,2.916729,2.990077,73.412752,72.353448,...,1.266467,62.215094,5.03,206.919255,0.409373,0.569918,387.257639,2020,59.9337,-39.47378
2020-01-22 07:11:00.006928,0.078067,-0.018678,282.610001,2491.938347,997.989629,999.240002,3.011154,3.093219,83.999203,82.909322,...,1.273664,99.670435,4.80,196.948117,0.498005,0.627132,387.299306,2020,59.9337,-39.47378


# Import normalized SAR data and params

In [12]:
data_dir = "/lustre/storeB/project/IT/geout/machine-ocean/data_raw/sentinel/"

In [13]:
with open(data_dir + 'in_situ_obs_with_sar_params.pickle', 'rb') as handle:
        sar_dict = pickle.load(handle)

In [14]:
sar_dict['Endurance_8'].keys()

dict_keys(['datetime_start', 'datetime_end', 'type', 'lat', 'lon', 'nc_data_location', 'products'])

# Merge in situ obs and SAR data

empty dict for new dfs

For each key in d:

    get buoy name
    get df for that buoy
    add empty cols for s0, inc, az
    for each product
        find closest timestamp in df to (beginposition+(endposition-startposition))/2
        fill in that row with sar params
    remove rows with nans
    save df with matches in new dict


In [15]:
dt = datetime.datetime(2019, 8, 11, 0, 10, 49, 874000)
#dt + dt

In [16]:
idx = df_in_situ.index.get_indexer([dt], method='nearest')[0]
idx

0

In [17]:
timestamp = df_in_situ.index[idx]
timestamp

Timestamp('2019-08-11 00:10:59.996491')

In [18]:
buoys

['Endurance_7',
 'SPURS1',
 'Endurance_3',
 'Pioneer_6',
 'Endurance_8',
 'Pioneer_8',
 'Irminger_3',
 'SPURS2',
 'SouthernOcean2',
 'Pioneer_5',
 'Pioneer_3',
 'Pioneer_7',
 'Endurance_4',
 'CLIMODE',
 'Irminger_7',
 'Endurance_9',
 'Pioneer_10',
 'Irminger_6']

epsilon = 5e-05
count = 0
collocated_dict = {}
for key, df in d.items():
    buoy = buoys[count]
    df['s0_norm'] = np.nan
    df['s0'] = np.nan
    df['inc'] = np.nan
    df['az'] = np.nan
    df_in_situ = d[key]
    df_in_situ['datetime'] = pd.to_datetime(df_in_situ['datetime'])
    df_in_situ = df_in_situ.set_index('datetime')
    
    for product in sar_dict[buoy]['products']:
        beginposition = sar_dict[buoy]['products'][product]['beginposition']
        endposition = sar_dict[buoy]['products'][product]['endposition']
        #sar_time = (beginposition + (endposition - startposition))/2  # 
        #TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'datetime.datetime'
        
        try:
            s0 = sar_dict[buoy]['products'][product]['sar_params'][str(epsilon)]['s0']
            s0_norm = sar_dict[buoy]['products'][product]['sar_params'][str(epsilon)]['s0_norm']
            inc = sar_dict[buoy]['products'][product]['sar_params'][str(epsilon)]['inc']
            az = sar_dict[buoy]['products'][product]['sar_params'][str(epsilon)]['az']

            # Find closest timestamp
            idx = df_in_situ.index.get_indexer([beginposition], method='nearest')[0]
            timestamp = df_in_situ.index[idx]

            # Fill in data
            df_in_situ.loc[df_in_situ.index==timestamp, 's0'] = s0
            df_in_situ.loc[df_in_situ.index==timestamp, 's0_norm'] = s0_norm
            df_in_situ.loc[df_in_situ.index==timestamp, 'inc'] = inc
            df_in_situ.loc[df_in_situ.index==timestamp, 'az'] = az
            print(buoy, s0)
        except:
            df_in_situ = pd.DataFrame()
            df_in_situ['s0'] = np.nan
            #print('File ', sar_dict[buoy]['products'][product]['filename'], ' does not exist')
    df_in_situ = df_in_situ.dropna(subset=['s0'])
    collocated_dict[buoy] = df_in_situ
    print(count)             
    count = count + 1

In [28]:
epsilon = 5e-04
count = 0
collocated_dict = {}
for key, df in d.items():
    buoy = buoys[count]
    df['s0_norm'] = np.nan
    df['s0'] = np.nan
    df['inc'] = np.nan
    df['az'] = np.nan
    df_in_situ = d[key]
    df_in_situ['datetime'] = pd.to_datetime(df_in_situ['datetime'])
    df_in_situ = df_in_situ.set_index('datetime')
    
    for product in sar_dict[buoy]['products']:
        beginposition = sar_dict[buoy]['products'][product]['beginposition']
        endposition = sar_dict[buoy]['products'][product]['endposition']
        #sar_time = (beginposition + (endposition - startposition))/2  # 
        #TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'datetime.datetime'
        if str(epsilon) in sar_dict[buoy]['products'][product]['sar_params'] :
            s0 = sar_dict[buoy]['products'][product]['sar_params'][str(epsilon)]['s0']
            s0_norm = sar_dict[buoy]['products'][product]['sar_params'][str(epsilon)]['s0_norm']
            inc = sar_dict[buoy]['products'][product]['sar_params'][str(epsilon)]['inc']
            az = sar_dict[buoy]['products'][product]['sar_params'][str(epsilon)]['az']

            # Find closest timestamp
            idx = df_in_situ.index.get_indexer([beginposition], method='nearest')[0]
            timestamp = df_in_situ.index[idx]

            # Fill in data
            df_in_situ.loc[df_in_situ.index==timestamp, 's0'] = np.nanmean(s0)
            df_in_situ.loc[df_in_situ.index==timestamp, 's0_norm'] = np.nanmean(s0_norm)
            df_in_situ.loc[df_in_situ.index==timestamp, 'inc'] = np.nanmean(inc)
            df_in_situ.loc[df_in_situ.index==timestamp, 'az'] = np.nanmean(az)
            print(buoy, np.nanmean(s0))
            if np.nanmean(s0)==-np.inf:
                print(sar_dict[buoy]['products'][product]['filename'])
                print(np.nanmean(s0))
                print(s0)
        #else:
        #    df_in_situ = pd.DataFrame()
        #    df_in_situ['s0'] = np.nan
            #print('File ', sar_dict[buoy]['products'][product]['filename'], ' does not exist')
    
    df_in_situ = df_in_situ.replace([-np.inf], np.nan)
    df_in_situ = df_in_situ.dropna(subset=['s0'])
    collocated_dict[buoy] = df_in_situ
    print(count)             
    count = count + 1

Endurance_7 -21.063286
Endurance_7 -22.216413
Endurance_7 -21.339077
Endurance_7 -25.85487
Endurance_7 -26.374071
Endurance_7 -24.314722
Endurance_7 -25.329073
Endurance_7 -28.75428
Endurance_7 -18.81792
Endurance_7 -17.72778
Endurance_7 -22.867931
Endurance_7 -25.324114
Endurance_7 -18.858349
Endurance_7 -21.884426
Endurance_7 -15.513192
Endurance_7 -22.038048
Endurance_7 -14.30343
Endurance_7 -19.869596
Endurance_7 -23.493656
Endurance_7 -16.396244
Endurance_7 -20.533834
Endurance_7 -19.869596
Endurance_7 -20.528923
Endurance_7 -21.946009
Endurance_7 -16.376326
Endurance_7 -16.2119
Endurance_7 -15.423333
Endurance_7 -11.019015
0
1
Endurance_3 -24.617025
Endurance_3 -20.701244
Endurance_3 -24.44478
Endurance_3 -19.07718
Endurance_3 -26.001904
Endurance_3 -19.006102
Endurance_3 -24.121826
Endurance_3 -15.626734
Endurance_3 -23.725256
Endurance_3 -19.326546
2
Pioneer_6 -8.744681
Pioneer_6 -22.633314
Pioneer_6 -20.283642
Pioneer_6 -15.981173
Pioneer_6 -19.310995
Pioneer_6 -13.40085
Pione

In [25]:
collocated_dict['Pioneer_7'][['UWr', 'rhoair', 's0']].index.get_indexer([datetime.datetime(2017, 10, 22, 22, 42, 24, 875000)], method='nearest')

array([15])

In [26]:
collocated_dict['Pioneer_7'][['UWr', 'rhoair', 's0']]

,UWr,rhoair,s0
datetime,,,
2017-06-12 11:10:59.967536,-0.042344,1.212813,-15.080737
2017-06-12 23:10:59.983629,-0.036690,1.213003,-21.165356
2017-06-24 23:10:59.998837,-0.041088,1.192970,-17.952158
2017-07-06 11:10:59.995377,-0.019187,1.203083,-18.854748
2017-07-18 23:10:59.995096,-0.015328,1.179373,-23.026979
2017-07-30 11:11:00.010425,-0.094400,1.197105,-15.889144
2017-07-30 23:11:00.001312,-0.066846,1.201028,-18.085205
2017-08-11 23:12:00.026565,0.000501,1.190140,-26.397457
2017-08-23 11:11:00.008795,-0.133680,1.166016,-16.625086


In [31]:
collocated_dict['Irminger_3'][['UWr', 'rhoair', 's0']]

,UWr,rhoair,s0
datetime,,,
2016-07-12 09:10:59.978919,-0.009191,1.253068,-21.923150
2016-07-12 20:10:59.999230,-0.000924,1.255386,-17.854439
2016-07-17 09:10:59.980719,-0.054515,1.241977,-16.119089
2016-07-17 20:10:59.974647,-0.017296,1.244527,-23.556729
2016-07-19 20:10:59.998415,0.014171,1.252602,-23.661248
...,...,...,...
2017-08-07 20:12:00.001057,-0.054866,1.252131,-8.296983
2017-08-11 09:11:59.999535,-0.041067,1.247842,-15.598502
2017-08-12 09:12:00.020667,-0.001397,1.244409,-23.947223


with open(data_dir + 'collocated_dict_5e_4.pickle', 'wb') as handle:
    pickle.dump(collocated_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
sar_dict['Pioneer_7']['products']['7e1d1641-af53-4486-bb45-8411c61d1aba']['sar_params']['5e-05'].keys()

dict_keys(['epsilon', 's0', 's0_norm', 'inc', 'az', 'grid_lons', 'grid_lats', 'pol'])

In [48]:
collocated_dict['Irminger_7']['s0'].shape

(0,)